In [ ]:
import os
import time
import numpy as np
from matplotlib import pyplot as plt
import cv2 as cv
import mediapipe as mp
import seaborn as sns
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras import models
from tensorflow.keras import layers
from sklearn.metrics import confusion_matrix ,classification_report
from gtts import gTTS
from playsound import playsound

In [ ]:
tf.__version__

In [ ]:
tf.test.is_gpu_available('gpu')

In [ ]:
input_types = ['palm', 'fist', 'thumbsup', 'gun', 'call']

In [ ]:
description_types = ['01', '02', '03', '04', '05']

In [ ]:
defination_types = ['a', 'b', 'c', 'd', 'e']

In [ ]:
cnn = models.load_model("cnn.h5")

In [ ]:
cnn.summary()

In [ ]:
l = []
for i in range(5):
    image_matrix = plt.imread('Inputs/call/' + 'call_' + str(i + 1) + '.jpg')
    l.append(image_matrix)
l = np.array(l)
l = l / 255

In [ ]:
cnn.predict(l)

In [ ]:
l.shape

In [ ]:
def Predict(img):
    class_ = np.argmax(cnn.predict(img))
    return input_types[class_]

In [ ]:
Predict(np.array([l[0]]))

In [ ]:
def saving_sounds():
    language = 'en'
    for types in input_types:
        obj = gTTS(text = types, lang = language, slow = False)
        if os.path.isfile(str(types) + ".mp3") is False:
            obj.save(str(types) + ".mp3")

In [ ]:
saving_sounds()

In [ ]:
types = 'thumbsup'
playsound(str(types) + ".mp3")

In [ ]:
def playing_sounds():
    for types in input_types:
        playsound(str(types) + ".mp3")

In [ ]:
playing_sounds()

In [ ]:
def message_class(mirror1, class_):
    cv.putText(mirror1, str(class_), (100, 100), cv.FONT_HERSHEY_PLAIN, 2, (255,0,0), 1)
    return mirror1

In [ ]:
def message_sound(class_):
    # if os.path.isfile(str(class_) + ".mp3") is False:
    # obj.save(str(class_) + ".mp3")
    playsound(str(class_) + ".mp3")

In [ ]:
message_sound("palm")

In [ ]:
def message_description(black1, class_):
    text1 = description_types[input_types.index(class_)]
    cv.putText(black1, str(text1), (100, 100), cv.FONT_HERSHEY_PLAIN, 5, (255,0,0), 1)
    return black1

In [ ]:
def message_defination(black2, class_):
    text2 = defination_types[input_types.index(class_)]
    cv.putText(black2, str(text2), (100, 100), cv.FONT_HERSHEY_PLAIN, 5, (255,0,0), 1)
    return black2

In [ ]:
mp_drawing = mp.solutions.drawing_utils
mp_holistic = mp.solutions.holistic

In [ ]:
holistic = mp_holistic.Holistic()

In [ ]:
R = 25
thickness = 2
webcam = 0
capture = cv.VideoCapture(webcam)
pre_class = ''
fps = int(capture.get(cv.CAP_PROP_FPS))
print("fps is " + str(fps))
_, frame = capture.read()
height, width, channel = frame.shape
while capture.isOpened():
    if cv.waitKey(1) & 0xFF == 13:
        break
    black = np.zeros(shape = frame.shape)
    black1 = np.zeros(shape = frame.shape)
    black2 = np.zeros(shape = frame.shape)
    _, frame = capture.read()
    frame_rgb = cv.cvtColor(frame, cv.COLOR_BGR2RGB)
    result = holistic.process(frame_rgb)
    try:
        hand_landmarks = result.right_hand_landmarks.landmark
        if hand_landmarks:
            x_max = 0
            y_max = 0
            x_min = width
            y_min = height
            for i in range(0,21,1):
                lm = hand_landmarks[i]
                x, y = int(lm.x * width), int(lm.y * height)
                if x > x_max:
                    x_max = x
                if x < x_min:
                    x_min = x
                if y > y_max:
                    y_max = y
                if y < y_min:
                    y_min = y
            frame_bgr = cv.cvtColor(frame_rgb, cv.COLOR_RGB2BGR)
            mp_drawing.draw_landmarks(frame_bgr, result.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS)
            cv.rectangle(frame_bgr, (x_min - R, y_min - R), (x_max + R, y_max + R), (0, 255, 0), thickness)
            result1 = frame_bgr
            mirror1 = cv.flip(result1, 1)
            mp_drawing.draw_landmarks(black, result.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS)
            croped = black[y_min - R + thickness: y_max + R - thickness, x_min - R + thickness : x_max + R - thickness]
            resized = cv.resize(croped, (96, 96))
            mirror2 = cv.flip(resized, 1)
            result2 = mirror2
            img_mat = np.array([result2])
            class_ = Predict(img_mat)
            # msg 01
            message_class(mirror1, class_)
            #msg 02
            # if pre_class != class_:
            # message_sound(class_)
            # pre_class = class_
            #msg 03
            black1 = message_description(black1, class_)
            result3 = black1
            #msg 04
            black2 = message_defination(black2, class_)
            result4 = black2
            cv.imshow("Frame2", result2)
            cv.imshow("Frame3", result3)
            cv.imshow("Frame4", result4)
    except:
        result1 = frame
        mirror1 = cv.flip(result1, 1)
        pass
    cv.imshow('frame1', mirror1)
capture.release()
cv.destroyAllWindows()

In [ ]:
fps is 30